# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [49]:
#Your code here
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns
from scipy.stats import norm

In [50]:
df = pd.read_csv('homepage_actions.csv')

In [51]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [52]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [54]:
#How many viewers also clicked
viewers_clicked = df[df['action'] == 'view']['id'].isin(df[df['action'] == 'click']['id']).sum()
viewers_clicked 

1860

In [70]:
#clicks_without_views
clicks_without_views = df[df['action'] == 'click']['id'].isin(~df[df['action'] == 'view']['id']).sum()
clicks_without_views

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

### Null Hypothesis (H0): There is no significant difference in the proportion of clicks between the experimental and control homepage versions.

### Alternative Hypothesis (Ha): The experimental homepage has a higher proportion of clicks than the control homepage.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [56]:
control_group = df[df['group'] == 'control']
control_group

,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view
...,...,...,...,...
8178,2017-01-18 08:17:12.675797,616692,control,view
8181,2017-01-18 09:07:37.661143,795585,control,view
8182,2017-01-18 09:09:17.363917,795585,control,click
8186,2017-01-18 10:08:51.588469,505451,control,view


In [57]:
experiment_group= df[df['group'] == 'experiment']
experiment_group

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
...,...,...,...,...
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view


In [58]:
#Your code here
#click-through rate for the control group
clicks_control = control_group[control_group['action'] == 'click'].shape[0]
visitors_control = control_group.shape[0]
control_click_through_rate = clicks_control / visitors_control
control_click_through_rate 

0.21857410881801126

In [59]:
expected_clicks_experiment = experiment_group.shape[0] * control_click_through_rate
expected_clicks_experiment

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [60]:
n_experiment = experiment_group.shape[0]
n_experiment 

3924

In [61]:
# Actual number of clicks in the experiment group
actual_clicks_experiment = experiment_group[experiment_group['action'] == 'click'].shape[0]
actual_clicks_experiment

928

In [62]:
variance_experiment = n_experiment * control_click_through_rate * (1 - control_click_through_rate)
variance_experiment 

670.2171115389895

In [63]:
std_dev_experiment = np.sqrt(variance_experiment)
std_dev_experiment 

25.888551746650283

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [64]:
#z-score
z_score = (actual_clicks_experiment - expected_clicks_experiment) / std_dev_experiment
z_score

2.71607302278784

In [66]:
#p-value 
p_value = 2 * (1 - norm.cdf(abs(z_score)))
p_value

0.0066061345518531045

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p-value (0.0066) is less than α (0.05),the null hypothesis is rejected in favor of the alternative hypothesis. So there is significant difference between the experimental and control groups in terms of click-through rate.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.